In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import pandas as pd 

In [2]:
# Webscraping process

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome('/usr/local/bin/chromedriver')
driver.get('https://www.gurufocus.com/insider/summary')

# Set the page size to 100 to reduce page loads
driver.find_element_by_xpath("//span[contains(text(),'40 / Page')]").click()
wait = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((
        By.XPATH,
        "//div[contains(text(),'60')]"))
)
element = driver.find_element_by_xpath("//div[contains(text(),'60')]").click()

# Wait for the page to load and don't overload the server
time.sleep(2)

# select Industry
driver.find_element_by_xpath("//span[contains(text(),'Industry')]").click()

# Select Industries
element = WebDriverWait(driver, 5).until(
    EC.presence_of_element_located((
        By.XPATH,
        "//span[contains(text(),'Financial Services')]")) #replace with industries like 'Financial Services', 'Semiconductors', 'Energy', etc
)
element.click()

# Use beautiful soup to scrap data from the page and form lists
ticker = []
company = []
price1 = []
insidername = []
insiderposition = []
date = []
buysell = []
tradingshare = []
sharechange = []
price2 = []
cost = []
finalshare = []
pricechange = []
dividend = []
pe = []
marketcap = []


time.sleep(3)
soup = BeautifulSoup(driver.page_source, 'html.parser')
tds = soup.select('td[data-column = "Price"]')
for td_col1, td_col2 in zip(tds[::2], tds[1::2]):
    price1.append(td_col1.text)
    price2.append(td_col2.text)
for tk in soup.find_all('td',{'class': 'table-stock-info', 'data-column': 'Ticker'}):
    ticker.append(tk.text)
for com in soup.find_all('td',{'data-column': 'Company'}):
    company.append(com.text)
for insiname in soup.find_all('td',{'data-column': 'Insider Name'}):
    insidername.append(insiname.text)
for inpos in soup.find_all('td',{'data-column': 'Insider Position'}):
    insiderposition.append(inpos.text)
for dat in soup.find_all('td',{'class': 'table-date-info', 'data-column': 'Date'}, ):
    date.append(dat.text)
for bs in soup.find_all('td',{'class':'table-type-info','data-column': 'Buy/Sell'}):
    buysell.append(bs.text)
for tradsh in soup.find_all('td',{'class':'right-align number-field', 'data-column': 'Insider Trading Shares'}):
    tradingshare.append(tradsh.text)
for shch in soup.find_all('td',{'class':'right-align number-field', 'data-column': 'Shares Change'}):
    sharechange.append(shch.text)
for cos in soup.find_all('td',{'class':'right-align number-field', 'data-column': 'Cost(000)'}):
    cost.append(cos.text)
for final in soup.find_all('td',{'class':'right-align number-field', 'data-column': 'Final Share'}):
    finalshare.append(final.text)
for prch in soup.find_all('td',{'class': 'right-align number-field', 'data-column': 'Price Change Since Insider Trade (%)'}):
    pricechange.append(prch.text)
for divid in soup.find_all('td',{'class': 'right-align number-field', 'data-column': 'Dividend Yield %'}):
    dividend.append(divid.text)
for per in soup.find_all('td',{'class': 'right-align number-field', 'data-column': 'PE Ratio'}):
    pe.append(per.text)
for marc in soup.find_all('td',{'class': 'right-align number-field', 'data-column': 'Market Cap ($M)'}):
    marketcap.append(marc.text)

In [3]:
#Create a dataframe based on the information collected

insidertrading = pd.DataFrame({"Ticker": ticker, "Company": company, "Insider Name": insidername, "Price": price1, "Insider Position": insiderposition,
             "Date": date, "Buy/Sell": buysell, "Insider Trading Shares": tradingshare, "shares Change": sharechange,"Price2": price2,
             "Cost(000)": cost, "Final Share": finalshare, "Price Change Since Insider Trade(%)": pricechange, "Dividend Yield %": dividend,
             "PE Ratio": pe, "Market Cap ($M)": marketcap})
insidertrading['Insider Name'] = insidertrading['Insider Name'].str.replace('\n', '')
insidertrading['Date'] = insidertrading['Date'].str.replace('\n', '')
insidertrading.head(10)

,Ticker,Company,Insider Name,Price,Insider Position,Date,Buy/Sell,Insider Trading Shares,shares Change,Price2,Cost(000),Final Share,Price Change Since Insider Trade(%),Dividend Yield %,PE Ratio,Market Cap ($M)
0,MCC,Medley Capital Corp,Fortress Investment Group LLC,$16.66,10% Owner,2020-08-11,Sell,"2,560",-0.93%,$16.43,$42.06,"272,781",1.40%,0,0,45.38
1,BKCC,BlackRock Capital Investment Corp,Pungello Michael,$2.91,Interim CFO/Interim Treasurer,2020-08-11,Buy,"8,000",22.30%,$2.94,$23.52,"43,872.38",-1.02%,17.87,0,204.20
2,CCB,Coastal Financial Corp,Lane Thomas D,$14.50,Director,2020-08-11,Buy,"17,000",16.96%,$13.93,$236.81,"117,239",4.09%,0,13.05,172.94
3,PBHC,Pathfinder Bancorp Inc,Rusnak Walter,$9.91,"Senior Vice President, CFO",2020-08-11,Buy,"1,000",2.45%,$9.15,$9.15,"41,773",8.31%,2.42,8.70,47.11
4,PACQ,Pure Acquisition Corp,WOODARD RODNEY L,$10.62,COO,2020-08-11,Buy,"2,000",173.91%,$10.61,$21.22,"3,150",0.09%,0,93.98,163.16
5,VIRT,Virtu Financial Inc,Cifu Douglas A,$23.59,CEO,2020-08-11,Buy,"100,000",51.88%,$22.45,"$2,245","292,755",5.08%,4.08,7.42,"4,557.12"
6,TSC,TriState Capital Holdings Inc,Fetterolf Brian S,$15.06,"CEO, TSC Bank, Director TSC",2020-08-11,Buy,"1,000",4.15%,$15.10,$15.10,"25,084",-0.26%,0,9.29,449.65
7,HGBL,Heritage Global Inc,DOVE ROSS,$3.03,CEO,2020-08-11,Buy,"7,225",0.40%,$3.22,$23.26,"1,809,015",-5.90%,0,27.30,89.15
8,FUSB,First US Bancshares Inc,BRIGGS ROBERT S,$6.44,Director,2020-08-11,Buy,"2,000",6.71%,$6.25,$12.50,"31,828",3.04%,1.70,12.15,39.77
9,CNNB,Cincinnati Bancorp Inc,Bedinghaus Robert A.,$8.63,Executive Chairman,2020-08-11,Buy,"1,000",1.28%,$8.68,$8.68,"79,321",-0.58%,0,23.97,25.68


In [4]:
#export dataframe to excel
insidertrading.to_excel('trading.xlsx') #will be downloaded in the same path of the notebook file if path not specified